In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
# import QUANTAXIS as QA
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [4]:
comp = pd.read_csv('data/fundamental.zip', 
                   parse_dates=['datadate'], infer_datetime_format=True)
comp = comp.drop(['indfmt', 'consol', 'popsrc', 'datafmt'], axis=1)

In [13]:
comp.drop()

,gvkey,datadate,fyearq,fqtr,tic,cusip,conm,curcdq,datacqtr,datafqtr,dlttq,oiadpq,pstkq,exchg,costat,mkvaltq
0,1000,1973-03-31,1973,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q1,1973Q1,7.000,NaN,0.000,12.0,I,NaN
4,1000,1974-03-31,1974,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1974Q1,1974Q1,7.000,NaN,0.000,12.0,I,NaN
8,1000,1975-03-31,1975,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1975Q1,1975Q1,7.000,NaN,0.414,12.0,I,NaN
12,1000,1976-03-31,1976,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1976Q1,1976Q1,10.114,1.719,2.069,12.0,I,NaN
16,1000,1977-03-31,1977,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1977Q1,1977Q1,15.350,1.381,1.569,12.0,I,NaN
20,1000,1978-03-31,1978,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1978Q1,1978Q1,17.779,0.753,0.000,12.0,I,NaN
21,1001,1983-03-31,1983,1,AMFD.,000165100,A & M FOOD SERVICES INC,USD,1983Q1,1983Q1,NaN,0.110,NaN,14.0,I,NaN
25,1001,1984-03-31,1984,1,AMFD.,000165100,A & M FOOD SERVICES INC,USD,1984Q1,1984Q1,4.198,0.351,0.000,14.0,I,NaN
29,1001,1985-03-31,1985,1,AMFD.,000165100,A & M FOOD SERVICES INC,USD,1985Q1,1985Q1,4.079,0.579,0.000,14.0,I,NaN
33,1001,1986-03-31,1986,1,AMFD.,000165100,A & M FOOD SERVICES INC,USD,1986Q1,1986Q1,11.411,0.596,0.000,14.0,I,NaN


按照gvkey, fyear, fqtr==1来进行resample;
groupby肯定是按照gvkey来进行的, 然后再按照fyear进行resample到year, 

In [ ]:
pd.cut()

In [6]:
crsp_name = pd.read_csv('data/crsp_name.csv', 
                   parse_dates=['namedt','nameendt'], infer_datetime_format=True)

In [7]:
crsp_name.head()

,Unnamed: 0,permno,shrcd,exchcd,namedt,nameendt
0,0,10000.0,10.0,3.0,1986-01-07,1986-12-03
1,1,10000.0,10.0,3.0,1986-12-04,1987-03-09
2,2,10000.0,10.0,3.0,1987-03-10,1987-06-11
3,3,10001.0,11.0,3.0,1986-01-09,1993-11-21
4,4,10001.0,11.0,3.0,1993-11-22,2004-06-09
